In [ ]:
import  torch, os
import  numpy as np
from    MiniImagenet import MiniImagenet
import  scipy.stats
from    torch.utils.data import DataLoader
from    torch.optim import lr_scheduler
import  random, sys, pickle
from    learner import Learner
import  argparse
import time
from attack import PGD


from torch.utils.tensorboard import SummaryWriter

torch.manual_seed(222)
torch.cuda.manual_seed_all(222)
np.random.seed(222)

device = torch.device('cude:2')
writer = SummaryWriter('./runs/size/PGD', comment='PGD')

#writer = SummaryWriter('./runs/size/aRUB', comment='aRUB')
# resize [28, 56, 84, 112, 140, 168, 196, 224] - 8번
config = [
        ('conv2d', [32, 3, 3, 3, 1, 0]),
        ('relu', [True]),
        ('bn', [32]),
        ('max_pool2d', [2, 2, 0]),
        ('conv2d', [32, 32, 3, 3, 1, 0]),
        ('relu', [True]),
        ('bn', [32]),
        ('max_pool2d', [2, 2, 0]),
        ('conv2d', [32, 32, 3, 3, 1, 0]),
        ('relu', [True]),
        ('bn', [32]),
        ('max_pool2d', [2, 1, 0]),
        ('flatten', []),
        ('linear', [args.n_way, 32 * 2 * 2])
]

for sz in [28, 56, 84, 112, 140, 168, 196, 224]:
    net = Learner(config, 3, sz).to(device)
    at = PGD(eps=eps / 255.0, sigma=2 / 255.0, nb_iter=10, DEVICE=device)
    #at = aRUB
    mini = MiniImagenet('route', mode='train', n_way=4, k_shot=25, k_query=1, batchsz=1, resize=sz)
    db = DataLoader(mini, 1, shuffle=True, num_workers=0, pin_memory=True)
    for step, (x_spt, y_spt, x_qry, y_qry) in enumerate(db):
        x_spt, y_spt, x_qry, y_qry = x_spt.to(device), y_spt.to(device), x_qry.to(device), y_qry.to(device)
        t = time.perf_counter()
        adv_inp = at.attack(net, net.parameters(), x_spt, y_spt)
        writer.add_scalar("time", time.perf_counter()-t)
        
    delete net

# adversarial sample 생성, 시간 측정

# 각 100개에 대해 수행 후 그래프 그리기